# Modeling Exercises

- Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage

1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

from env import username, password, host
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import statsmodels.api as sm
from statsmodels.tsa.api import Holt, ExponentialSmoothing
np.random.seed(0)

from sklearn.metrics import mean_squared_error
from math import sqrt

import acquire

In [95]:
df = pd.read_csv('saas.csv')

In [96]:
df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [97]:
df['Month_Invoiced'] = pd.to_datetime(df['Month_Invoiced'])

In [99]:
df = df.set_index('Month_Invoiced').sort_index()

In [100]:
df.dtypes

Customer_Id            int64
Invoice_Id             int64
Subscription_Type    float64
Amount               float64
dtype: object

In [101]:
df.shape

(790458, 4)

In [102]:
train_size = int(round(df.shape[0] * 0.5))
validate_size = int(round(df.shape[0] * 0.3))
test_size = int(round(df.shape[0] * 0.2))

In [103]:
train_size, validate_size, test_size

(395229, 237137, 158092)

In [104]:
len(df) == train_size + validate_size + test_size

True

In [105]:
validate_end_index = train_size + validate_size

In [106]:
validate_end_index

632366

In [107]:
train = df[:train_size]
validate = df[train_size:validate_end_index]
test = df[validate_end_index:]

In [108]:
train.shape[0], validate.shape[0], test.shape[0]

(395229, 237137, 158092)

In [114]:
train.head()

,sub_type,amount
Month_Invoiced,,
2014-01-31,0.0,0.0
2014-01-31,2.0,10.0
2014-01-31,0.0,0.0
2014-01-31,0.0,0.0
2014-01-31,1.0,5.0


In [110]:
train = train.rename(columns={'Subscription_Type': 'sub_type',
                              'Amount': 'amount'})

In [111]:
train = train.drop(columns=['Customer_Id', 'Invoice_Id'])

In [112]:
validate = validate.rename(columns={'Subscription_Type': 'sub_type',
                              'Amount': 'amount'})
validate = validate.drop(columns=['Customer_Id', 'Invoice_Id'])

In [113]:
test = test.rename(columns={'Subscription_Type': 'sub_type',
                              'Amount': 'amount'})
test = test.drop(columns=['Customer_Id', 'Invoice_Id'])

In [69]:
def time_series_subset(df):
    '''
    
    '''
    train_size = int(round(df.shape[0] * 0.5))
    validate_size = int(round(df.shape[0] * 0.3))
    test_size = int(round(df.shape[0] * 0.2))
    
    print(train_size, validate_size, test_size)
    print(len(df) == train_size + validate_size + test_size)
    
    validate_end_index = train_size + validate_size
    
    train = df[:train_size]
    validate = df[train_size:validate_end_index]
    test = df[validate_end_index:]
    
    return train, validate, test



2. Forecast, plot and evaluate using each at least 4 of the methods we discussed:
- Last Observed Value

In [68]:
train.head()

,sub_type,amount
Month_Invoiced,,
2014-01-31,0.0,0.0
2014-01-31,2.0,10.0
2014-01-31,0.0,0.0
2014-01-31,0.0,0.0
2014-01-31,1.0,5.0


In [116]:
sub_type = train['sub_type'][-1:][0]

In [117]:
amount = train['amount'][-1:][0]

In [118]:
sub_type, amount

(0.0, 0.0)

In [119]:
yhat_df = pd.DataFrame(
    {'sub_type': [sub_type],
     'amount': [amount]},
    index=validate.index)

yhat_df.head()

,sub_type,amount
Month_Invoiced,,
2015-10-31,0.0,0.0
2015-10-31,0.0,0.0
2015-10-31,0.0,0.0
2015-10-31,0.0,0.0
2015-10-31,0.0,0.0


- Simple Average

In [120]:
avg_sub = round(train['sub_type'].mean(), 2)
avg_amount = round(train['amount'].mean(), 2)

In [121]:
yhat_df = make_baseline_predictions(avg_sub, avg_amount)
yhat_df.head()

,sub_type,amount
Month_Invoiced,,
2015-10-31,0.5,2.48
2015-10-31,0.5,2.48
2015-10-31,0.5,2.48
2015-10-31,0.5,2.48
2015-10-31,0.5,2.48


In [122]:
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])

In [123]:
for col in train.columns:
    eval_df = append_eval_df(model_type= 'simple_average', 
                            target_var= col)
eval_df

,model_type,target_var,rmse
0,simple_average,sub_type,1.0
1,simple_average,amount,5.0


- Moving Average

In [124]:
period=30

rolling_subs = round(train['sub_type'].rolling(period).mean()[-1], 2)
rolling_amounts = round(train['amount'].rolling(period).mean()[-1], 2)
print(rolling_subs, rolling_amounts)

0.57 2.83


In [125]:
yhat_df = make_baseline_predictions(rolling_subs, rolling_amounts)
yhat_df.head()

,sub_type,amount
Month_Invoiced,,
2015-10-31,0.57,2.83
2015-10-31,0.57,2.83
2015-10-31,0.57,2.83
2015-10-31,0.57,2.83
2015-10-31,0.57,2.83


- Holt's Linear Trend 

In [126]:
train.dtypes

sub_type    float64
amount      float64
dtype: object

In [133]:
for col in train.columns:
    
    sm.tsa.seasonal_decompose(train[col].resample('w').mean()).plot()

ValueError: This function does not handle missing values

- Holt's Seasonal Trend

- Based on previous year/month/etc., this is up to you.

Bonus: 
1. Using the store item demand data, create a forecast of `sales_total` and `quantity` for 2018 using the `Previous Cycle` approach.  .  
2. Predict 2018 total **monthly** sales for a single store and/or item by creating a model using prophet.
3. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
4. Plot the 2018 monthly sales predictions.

In [62]:
def plot_and_eval(target_var):
    '''
    This function takes in the target var name (string), and returns a plot
    of the values of train for that variable, validate, and the predicted values from yhat_df. 
    it will als lable the rmse. 
    '''
    plt.figure(figsize = (12,4))
    plt.plot(train[target_var], label='Train', linewidth=1, color='#377eb8')
    plt.plot(validate[target_var], label='Validate', linewidth=1, color='#ff7f00')
    plt.plot(yhat_df[target_var], label='yhat', linewidth=2, color='#a65628')
    plt.legend()
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.0f}'.format(rmse))
    plt.show()

In [63]:
def make_baseline_predictions(col1, col2):
    yhat_df = yhat_df = pd.DataFrame({'sub_type': [col1],
                                      'amount': [col2]},
                                       index=validate.index)
    return yhat_df

In [64]:
# function to store the rmse so that we can compare
def append_eval_df(model_type, target_var):
    '''
    this function takes in as arguments the type of model run, and the name of the target variable. 
    It returns the eval_df with the rmse appended to it for that model and target_var. 
    '''
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var],
        'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

In [65]:
def evaluate(target_var):
    '''
    This function will take the actual values of the target_var from validate, 
    and the predicted values stored in yhat_df, 
    and compute the rmse, rounding to 0 decimal places. 
    it will return the rmse. 
    '''
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])), 0)
    return rmse